In [3]:
# Install the necessary packages for working with datasets, transformers, TensorFlow, and evaluation tools.
!pip -q install datasets
!pip -q install transformers=='4.29.0'
!pip -q install tensorflow=='2.15'  # At least this TensorFlow version is required to use the "evaluate" module.
!pip -q install evaluate
!pip -q install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 44.2 MB/s eta 0:00:00


In [1]:
import os
import torch
import zipfile
import evaluate
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
from datasets import Dataset
from google.colab import drive
import matplotlib.pyplot as plt
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from huggingface_hub import notebook_login
from transformers import AutoImageProcessor
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms
from transformers import DefaultDataCollator
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

In [1]:
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def extract_zip_file(zip_path, dest_path):
    """
    Extracts a ZIP file to a specified directory.

    Args:
        zip_path (str): The file path to the ZIP file.
        dest_path (str): The destination directory where the contents will be extracted.

    This function opens the specified ZIP file in read mode and extracts all its contents
    to the given destination directory.
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(dest_path)  # Extract all files to the destination directory

# Usage
extract_zip_file('/content/drive/MyDrive/data.zip', 'data')


In [2]:
notebook_login()

In [ ]:
# Load the CSV files
y_train = pd.read_csv('/content/data/Y_train.csv')
x_train = pd.read_csv('/content/data/X_train.csv')
x_train = x_train.head(100)
y_train = y_train.head(100)

In [ ]:
# Make sure that the dataframes are aligned
assert len(y_train) == len(x_train), "The number of labels does not match the number of image paths."

In [ ]:
# Initialize lists to store the data
image_file_paths = []
images = []
labels = []

def load_images_and_labels(x_train, y_train):
    """
    Loads images and their corresponding labels, converts images to PyTorch tensors,
    and stores the file paths, tensors, and labels in respective lists.

    Args:
        x_train (pd.DataFrame): DataFrame containing image file paths.
        y_train (pd.DataFrame): DataFrame containing labels corresponding to the images.

    Returns:
        tuple: A tuple containing three lists - image file paths, images as tensors, and labels.
    """
    for i in range(len(y_train)):
        # Construct the full image file path
        image_file_path = '/content/' + x_train.loc[i, 'image_path']

        # Load the image using PIL
        image = Image.open(image_file_path)

        # Get the corresponding label
        label = y_train.loc[i, 'output']

        # Convert the image to a PyTorch tensor
        image = ToTensor()(image)
        image = torch.Tensor(image)  # Ensure the image is a PyTorch tensor

        # Check that the image has been correctly transformed to a tensor
        if not isinstance(image, torch.Tensor):
            print(f"Warning: Expected image to be a PyTorch tensor after transformation but got {type(image)}. Skipping this image.")
            continue

        # Append the data to the lists
        image_file_paths.append(image_file_path)
        images.append(image)
        labels.append(label)

    return image_file_paths, images, labels

# Example usage
image_file_paths, images, labels = load_images_and_labels(x_train, y_train)

In [ ]:
def create_dataset(image_file_paths, images, labels):
    """
    Creates a dataset from given lists of image file paths, images, and labels.

    Args:
        image_file_paths (list): List of file paths to the images.
        images (list): List of images as PyTorch tensors.
        labels (list): List of labels corresponding to the images.

    Returns:
        Dataset: A Hugging Face `Dataset` object containing the image file paths, images, and labels.
    """
    # Create the dataset from the provided lists
    dataset = Dataset.from_dict({
        'image_file_path': image_file_paths,
        'image': images,
        'labels': labels
    })
    return dataset

# Example usage
dataset_train = create_dataset(image_file_paths, images, labels)

In [ ]:
dataset_train

In [ ]:
def display_random_images(dataset, num_images=4, labels_names=None):
    """
    Displays a grid of random images from the dataset along with their labels.

    Args:
        dataset (Dataset): The dataset containing images and labels.
        num_images (int, optional): The number of images to display. Default is 4.
        labels_names (dict, optional): A dictionary mapping label indices to label names. Default is None.

    This function randomly selects a specified number of images from the dataset and
    displays them in a grid format using Matplotlib. The images are converted to
    PyTorch tensors if they are not already, and their dimensions are rearranged for display.
    """
    num_rows = 2
    num_cols = np.ceil(num_images / num_rows).astype(int)

    # Set up the figure size based on the number of columns and rows
    plt.figure(figsize=(num_cols * 3, num_rows * 3))

    # Randomly select indices from the dataset
    indices = np.random.choice(range(len(dataset)), size=num_images, replace=False)

    for i, idx in enumerate(indices, 1):
        idx = int(idx)
        image = dataset[idx]['image']
        label = dataset[idx]['labels']
        label_name = labels_names.get(label, "Unknown") if labels_names else label

        # Convert the image to a PyTorch tensor if it's a list
        if isinstance(image, list):
            image = torch.Tensor(image)

        if isinstance(image, torch.Tensor):
            # Rearrange the dimensions of the image for plotting
            image = image.permute(1, 2, 0)

            plt.subplot(num_rows, num_cols, i)
            plt.imshow(image)
            plt.title(f"{label_name} ({label})")
            plt.axis('off')

    plt.tight_layout()
    plt.show()

# Example usage
display_random_images(dataset_train, num_images=4, labels_names={0: 'NotQuarry', 1: 'Quarry'})


In [ ]:
id2label = {'0': 'NotQuarry', '1': 'Quarry'}
label2id = {'NotQuarry':'0' , 'Quarry':'1'}
labels = ['NotQuarry','Quarry']

In [ ]:
def split_dataset(dataset, test_size=0.2):
    """
    Splits the dataset into training and testing sets.

    Args:
        dataset (Dataset): The dataset to split.
        test_size (float, optional): The proportion of the dataset to include in the test split. Default is 0.2.

    Returns:
        DatasetDict: A dictionary containing the training and testing datasets.
    """
    # Split the dataset into training and testing sets
    split_dataset = dataset.train_test_split(test_size=test_size)
    return split_dataset

# Example usage
dataset_train = split_dataset(dataset_train, test_size=0.2)
print(dataset_train)

In [ ]:
def initialize_image_processor(checkpoint):
    """
    Initializes an image processor using a pre-trained model checkpoint.

    Args:
        checkpoint (str): The model checkpoint to use for the image processor.

    Returns:
        AutoImageProcessor: An initialized image processor.
    """
    # Load the pre-trained image processor from the specified checkpoint
    image_processor = AutoImageProcessor.from_pretrained(checkpoint)
    return image_processor

# Example usage
checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = initialize_image_processor(checkpoint)

In [ ]:
def create_image_transforms(image_processor):
    """
    Creates a composition of image transforms based on the specifications of an image processor.

    Args:
        image_processor (AutoImageProcessor): The image processor containing the mean, std, and size specifications.

    Returns:
        Compose: A composition of image transforms.
    """
    # Define the normalization transform using the image processor's mean and std values
    normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

    # Determine the size for the RandomResizedCrop transform
    size = (
        image_processor.size["shortest_edge"]
        if "shortest_edge" in image_processor.size
        else (image_processor.size["height"], image_processor.size["width"])
    )

    # Create a composition of transforms: RandomResizedCrop, ToTensor, and Normalize
    transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])
    return transforms

# Example usage
_transforms = create_image_transforms(image_processor)


In [ ]:
def transforms(examples):
    """
    Applies a series of image transformations to the images in the provided examples.

    Args:
        examples (dict): A dictionary containing the examples, where each example has an "image" key.

    Returns:
        dict: A dictionary with transformed images under the "pixel_values" key and the original "image" key removed.
    """
    transformed_images = []

    for img in examples["image"]:
        # Check if img is a PIL Image object
        if isinstance(img, Image.Image):
            # Convert image to RGB and apply transformations
            transformed_images.append(_transforms(img.convert("RGB")))
        else:
            # Print a warning if the image is not a PIL Image
            print(f"Warning: Expected img to be a PIL Image but got {type(img)}. Skipping this image.")

    # Add transformed images to examples with the key "pixel_values"
    examples["pixel_values"] = transformed_images
    # Remove the original "image" key from examples
    del examples["image"]

    return examples

# Apply the transforms function to the dataset
dataset_train = dataset_train.with_transform(transforms)


In [ ]:
def initialize_data_collator():
    """
    Initializes a default data collator for use in data loading and batching.

    Returns:
        DefaultDataCollator: An instance of DefaultDataCollator.
    """
    # Create an instance of DefaultDataCollator
    data_collator = DefaultDataCollator()
    return data_collator

def load_accuracy_metric():
    """
    Loads the accuracy metric for evaluation.

    Returns:
        Metric: An accuracy metric object.
    """
    # Load the accuracy metric using the evaluate library
    accuracy = evaluate.load("accuracy")
    return accuracy

# Example usage
data_collator = initialize_data_collator()
accuracy = load_accuracy_metric()

In [ ]:
def compute_metrics(eval_pred):
    """
    Computes evaluation metrics for model predictions.

    Args:
        eval_pred (tuple): A tuple containing the model predictions and the true labels.
                           - predictions (numpy.ndarray): The model's output predictions.
                           - labels (numpy.ndarray): The true labels for the evaluation dataset.

    Returns:
        dict: A dictionary containing the computed accuracy metric.
    """
    predictions, labels = eval_pred

    # Get the index of the highest predicted value for each prediction (the predicted class)
    predictions = np.argmax(predictions, axis=1)

    # Compute the accuracy metric using the predictions and the true labels
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
def initialize_model(checkpoint, num_labels, id2label, label2id):
    """
    Initializes a pre-trained image classification model.

    Args:
        checkpoint (str): The model checkpoint to use for initialization.
        num_labels (int): The number of labels (classes) for the classification task.
        id2label (dict): A dictionary mapping label IDs to label names.
        label2id (dict): A dictionary mapping label names to label IDs.

    Returns:
        AutoModelForImageClassification: An initialized image classification model.
    """
    # Load the pre-trained image classification model with specified parameters
    model = AutoModelForImageClassification.from_pretrained(
        checkpoint,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id,
    )
    return model

# Example usage
checkpoint = "google/vit-base-patch16-224-in21k"
num_labels = len(labels)

model = initialize_model(checkpoint, num_labels, id2label, label2id)


In [ ]:
def create_training_arguments(output_dir, learning_rate=5e-5, train_batch_size=16, eval_batch_size=16, num_epochs=3, warmup_ratio=0.1, logging_steps=10):
    """
    Creates training arguments for model training.

    Args:
        output_dir (str): The directory where the model checkpoints and logs will be saved.
        learning_rate (float, optional): The learning rate for the optimizer. Default is 5e-5.
        train_batch_size (int, optional): The batch size per device for training. Default is 16.
        eval_batch_size (int, optional): The batch size per device for evaluation. Default is 16.
        num_epochs (int, optional): The number of training epochs. Default is 3.
        warmup_ratio (float, optional): The warmup ratio for the learning rate schedule. Default is 0.1.
        logging_steps (int, optional): The number of steps between logging. Default is 10.

    Returns:
        TrainingArguments: An instance of TrainingArguments with the specified parameters.
    """
    training_args = TrainingArguments(
        output_dir=output_dir,
        remove_unused_columns=False,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=train_batch_size,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=eval_batch_size,
        num_train_epochs=num_epochs,
        warmup_ratio=warmup_ratio,
        logging_steps=logging_steps,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy"
    )
    return training_args

# Example usage
training_args = create_training_arguments(
    output_dir="./ORIS_classification",
    learning_rate=5e-5,
    train_batch_size=16,
    eval_batch_size=16,
    num_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10
)

In [ ]:
def create_trainer(model, training_args, data_collator, train_dataset, eval_dataset, tokenizer, compute_metrics):
    """
    Creates a Trainer instance for training and evaluating the model.

    Args:
        model (PreTrainedModel): The pre-trained model to be trained and evaluated.
        training_args (TrainingArguments): The training arguments specifying the training configuration.
        data_collator (DataCollator): The data collator used for batching data.
        train_dataset (Dataset): The training dataset.
        eval_dataset (Dataset): The evaluation dataset.
        tokenizer (AutoImageProcessor): The tokenizer or image processor for preprocessing the data.
        compute_metrics (function): The function used to compute evaluation metrics.

    Returns:
        Trainer: An instance of Trainer configured with the provided parameters.
    """
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    return trainer

# Example usage
trainer = create_trainer(
    model=model,
    training_args=training_args,
    data_collator=data_collator,
    train_dataset=dataset_train["train"],
    eval_dataset=dataset_train["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

In [ ]:
#Pushing the model
trainer.model.push_to_hub("MohammedNasri/ORIS_classification")

In [ ]:
import os
import torch
import zipfile
import evaluate
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
from datasets import Dataset
from google.colab import drive
import matplotlib.pyplot as plt
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from huggingface_hub import notebook_login
from transformers import AutoImageProcessor
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms
from transformers import DefaultDataCollator
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
drive.mount('/content/drive')
def extract_zip_file(zip_path, dest_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(dest_path)

extract_zip_file('/content/drive/MyDrive/data.zip', 'data')
notebook_login()
y_train = pd.read_csv('/content/data/Y_train.csv')
x_train = pd.read_csv('/content/data/X_train.csv')
x_train = x_train.head(100)
y_train = y_train.head(100)
# Initialize lists to store the data
image_file_paths = []
images = []
labels = []

def load_images_and_labels(x_train, y_train):
    for i in range(len(y_train)):
        image_file_path = '/content/' + x_train.loc[i, 'image_path']
        image = Image.open(image_file_path)
        label = y_train.loc[i, 'output']
        image = ToTensor()(image)
        image = torch.Tensor(image)  
        if not isinstance(image, torch.Tensor):
            print(f"Warning: Expected image to be a PyTorch tensor after transformation but got {type(image)}. Skipping this image.")
            continue
        image_file_paths.append(image_file_path)
        images.append(image)
        labels.append(label)
    return image_file_paths, images, labels
image_file_paths, images, labels = load_images_and_labels(x_train, y_train)
def create_dataset(image_file_paths, images, labels):
    dataset = Dataset.from_dict({
        'image_file_path': image_file_paths,
        'image': images,
        'labels': labels
    })
    return dataset
dataset_train = create_dataset(image_file_paths, images, labels)
def display_random_images(dataset, num_images=4, labels_names=None):
    num_rows = 2
    num_cols = np.ceil(num_images / num_rows).astype(int)
    plt.figure(figsize=(num_cols * 3, num_rows * 3))
    indices = np.random.choice(range(len(dataset)), size=num_images, replace=False)
    for i, idx in enumerate(indices, 1):
        idx = int(idx)
        image = dataset[idx]['image']
        label = dataset[idx]['labels']
        label_name = labels_names.get(label, "Unknown") if labels_names else label
        if isinstance(image, list):
            image = torch.Tensor(image)
        if isinstance(image, torch.Tensor):
            image = image.permute(1, 2, 0)
            plt.subplot(num_rows, num_cols, i)
            plt.imshow(image)
            plt.title(f"{label_name} ({label})")
            plt.axis('off')

    plt.tight_layout()
    plt.show()

display_random_images(dataset_train, num_images=4, labels_names={0: 'NotQuarry', 1: 'Quarry'})
id2label = {'0': 'NotQuarry', '1': 'Quarry'}
label2id = {'NotQuarry':'0' , 'Quarry':'1'}
labels = ['NotQuarry','Quarry']
def split_dataset(dataset, test_size=0.2):
    split_dataset = dataset.train_test_split(test_size=test_size)
    return split_dataset
dataset_train = split_dataset(dataset_train, test_size=0.2)
def initialize_image_processor(checkpoint):
    image_processor = AutoImageProcessor.from_pretrained(checkpoint)
    return image_processor
checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = initialize_image_processor(checkpoint)
def create_image_transforms(image_processor):
    normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
    size = (
        image_processor.size["shortest_edge"]
        if "shortest_edge" in image_processor.size
        else (image_processor.size["height"], image_processor.size["width"])
    )
    transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])
    return transforms

# Example usage
_transforms = create_image_transforms(image_processor)
def transforms(examples):
    transformed_images = []
    for img in examples["image"]:
        if isinstance(img, Image.Image):
            transformed_images.append(_transforms(img.convert("RGB")))
        else:
            print(f"Warning: Expected img to be a PIL Image but got {type(img)}. Skipping this image.")

    examples["pixel_values"] = transformed_images
    del examples["image"]

    return examples

dataset_train = dataset_train.with_transform(transforms)

def initialize_data_collator():

    data_collator = DefaultDataCollator()
    return data_collator

def load_accuracy_metric():
    accuracy = evaluate.load("accuracy")
    return accuracy
data_collator = initialize_data_collator()
accuracy = load_accuracy_metric()
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)
def initialize_model(checkpoint, num_labels, id2label, label2id):
    model = AutoModelForImageClassification.from_pretrained(
        checkpoint,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id,
    )
    return model

# Example usage
checkpoint = "google/vit-base-patch16-224-in21k"
num_labels = len(labels)

model = initialize_model(checkpoint, num_labels, id2label, label2id)
def create_training_arguments(output_dir, learning_rate=5e-5, train_batch_size=16, eval_batch_size=16, num_epochs=3, warmup_ratio=0.1, logging_steps=10):
    training_args = TrainingArguments(
        output_dir=output_dir,
        remove_unused_columns=False,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=train_batch_size,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=eval_batch_size,
        num_train_epochs=num_epochs,
        warmup_ratio=warmup_ratio,
        logging_steps=logging_steps,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy"
    )
    return training_args

# Example usage
training_args = create_training_arguments(
    output_dir="./ORIS_classification",
    learning_rate=5e-5,
    train_batch_size=16,
    eval_batch_size=16,
    num_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10
)
def create_trainer(model, training_args, data_collator, train_dataset, eval_dataset, tokenizer, compute_metrics):
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    return trainer
trainer = create_trainer(
    model=model,
    training_args=training_args,
    data_collator=data_collator,
    train_dataset=dataset_train["train"],
    eval_dataset=dataset_train["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics
)
trainer.train()
trainer.model.push_to_hub("MohammedNasri/ORIS_classification")